In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.8 MB/s eta 0:0

In [2]:
import pandas as pd
import os
import sqlite3
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

In [5]:
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

In [6]:
def excel_to_db(filepath,database_name):
  '''load excel data from filepath and insert it to database_name'''
  #load excel file
  tables = pd.ExcelFile(filepath)

  #connect/create database_name
  db = sqlite3.connect(database_name)


  #insert data to sql database
  for sheet in tables.sheet_names:
    df = pd.read_excel(tables, sheet)
    df.to_sql(sheet, db,index=False, if_exists='replace')

  #end connection to database
  db.close()
  return tables



In [13]:
file_path = 'Xyris HIS_data.xlsx'
database = 'Xyris.db'

tables = excel_to_db(file_path, database)

In [8]:
def read_sql(db_table, database):
  df = pd.read_sql(f'SELECT * FROM {db_table};',database)
  return df

In [9]:
# vectorize the data

def vectorize_data(df,sheet_name):
  chunks= []
  metadatas = []
  ids = []
  for idx, row in df.iterrows():
    sentence = ''
    for col in df.columns:
      sentence += f'({col}: {row[col]}), \n'
    chunks.append(sentence)
    metadatas.append({'Source':sheet_name, 'index' : idx })
    ids.append(f'__{sheet_name}__{idx}')
  return chunks, metadatas, ids




## Setup ChromaDB

In [17]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient('chroma')

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name='all-MiniLM-L12-v2'
)

collection = client.get_or_create_collection(name= 'Xyris-HIS', embedding_function=sentence_transformer_ef)

In [18]:
def add_to_collection(collection, chunks, metadatas, ids):
  collection.add(
      documents= chunks,
      metadatas= metadatas,
      ids= ids
  )


In [11]:
# Function process tables
def process_excel(collection, tables, database_name):
  db = sqlite3.connect(database_name)
  chunks=[]
  for sheet in tables.sheet_names:
    df = read_sql(sheet, db)
    #vector data
    print(f'Preparing vectors for sheet: {sheet}')
    chunk , metadatas, ids = vectorize_data(df,sheet)
    chunks.append(chunk)
    print(f'Number of chunks {len(chunk)} vectorized')
    print('adding to collection...')
    add_to_collection(collection, chunk, metadatas, ids)
    print('added successfully')
  db.close()
  return chunks, metadatas, ids

In [19]:
chunks, metadatas, ids= process_excel(collection,tables, database)

Preparing vectors for sheet: Physicians
Number of chunks 25 vectorized
adding to collection...
added successfully
Preparing vectors for sheet: Schedules
Number of chunks 25 vectorized
adding to collection...
added successfully
Preparing vectors for sheet: Specialities
Number of chunks 25 vectorized
adding to collection...
added successfully
Preparing vectors for sheet: Pricelist
Number of chunks 60 vectorized
adding to collection...
added successfully
Preparing vectors for sheet: Policy
Number of chunks 1 vectorized
adding to collection...
added successfully


In [20]:
#get docs matches with the query
def semantic_search(collection, query, k):
  results= collection.query(query_texts= [query], n_results= k)
  return results

In [21]:
query = 'what is Dr. Alice\'s speciality'
semantic_search(collection, query, 3 )

{'ids': [['__Physicians__0', '__Schedules__0', '__Physicians__14']],
 'embeddings': None,
 'documents': [['(Name: Dr. Alice Smith), \n(Speciality: Cardiologist), \n(Degree: Specialist), \n',
   '(Doctor Name: Dr. Alice Smith), \n(Monday: 09:00-14:00), \n(Tuesday: 09:00-14:00), \n(Wednesday: Off), \n(Thursday: 09:00-14:00), \n(Friday: 09:00-14:00), \n(Saturday: 10:00-13:00), \n(Sunday: Off), \n',
   '(Name: Dr. Mary Scott), \n(Speciality: Family Physician), \n(Degree: Specialist), \n']],
 'uris': None,
 'data': None,
 'metadatas': [[{'Source': 'Physicians', 'index': 0},
   {'Source': 'Schedules', 'index': 0},
   {'Source': 'Physicians', 'index': 14}]],
 'distances': [[1.093923807144165, 1.1337804794311523, 1.1582534313201904]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [22]:
def get_context_with_sources(results):
    """Extract context and source information from search results"""
    # Combine document chunks into a single context
    context = "\n\n".join(results['documents'][0])

    return context

In [23]:
from openai import OpenAI

client = OpenAI()

def get_prompt(context: str, query: str):
    """Generate a prompt combining context, history, and query"""
    prompt = f"""Based on the following context and conversation history,
    please provide a relevant and contextual response. If the answer cannot
    be derived from the context, only use the conversation history or say
    "I cannot answer this based on the provided information."

    Context from documents:
    {context}


    Human: {query}

    Assistant:"""

    return prompt

def generate_response(query: str, context: str):
    '''Generate a response using OpenAI '''
    prompt = get_prompt(context,  query)


    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,  # Lower temperature for more focused responses
        max_tokens=500
    )
    return response.choices[0].message.content

In [24]:
def rag_query(collection, query: str, n_chunks: int = 2):
    """Perform RAG query: retrieve relevant chunks and generate answer"""
    # Get relevant chunks
    results = semantic_search(collection, query, n_chunks)
    context = get_context_with_sources(results)

    # Generate response
    response = generate_response(query, context)

    return response


In [25]:
# query = 'what is Dr. Alice\'s speciality'
response = rag_query(collection, query)

In [26]:
print(response)

Dr. Alice Smith's speciality is cardiology.
